In [51]:
textRDD = sc.newAPIHadoopFile('../Data/Moby-Dick.txt',
                              'org.apache.hadoop.mapreduce.lib.input.TextInputFormat',
                              'org.apache.hadoop.io.LongWritable',
                              'org.apache.hadoop.io.Text',
                               conf={'textinputformat.record.delimiter': "\r\n\r\n"}
                             ).map(lambda x: x[1])
    
sentences = textRDD.flatMap(lambda x: x.split(". "))    

def printOutput(n, freq_ngramRDD):
    top = freq_ngramRDD.take(5)
    print '\n============ %d most frequent %d-grams'%(5, n)
    print '\nindex\tcount\tngram'
    for i in xrange(5):
        print '%d.\t%d: \t"%s"' % (i + 1,top[i][0], ' '.join(top[i][1]))

import re, string

# setence level 
words = sentences.map(lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x)
        ).map(lambda x: filter(lambda x: x != '', x.lower().replace('\r\n', ' ').split(' ')))

def get_ngrams(lst, n):
    for i in xrange(len(lst)-n+1):  # edge condition 
        yield tuple(lst[i:i+n])  # joined at the final output. tuple here instead
        
def ngrams_stats(words, n):
    grams = words.flatMap(lambda x: get_ngrams(x, n))  # closure for n

    grams_stats = grams.map(lambda x: (x, 1)
            ).reduceByKey(lambda a, b: a + b)
    grams_reverse_stats = grams_stats.map(lambda p: (p[1], p[0])
            ).sortByKey(False)
    return grams_reverse_stats

In [52]:
for n in xrange(1,6):
    freq_ngramRDD = ngrams_stats(words, n)
    printOutput(n, freq_ngramRDD)


============ 5 most frequent 1-grams

index	count	ngram
1.	14620: 	"the"
2.	6732: 	"of"
3.	6502: 	"and"
4.	4799: 	"a"
5.	4706: 	"to"

============ 5 most frequent 2-grams

index	count	ngram
1.	1906: 	"of the"
2.	1193: 	"in the"
3.	746: 	"to the"
4.	444: 	"from the"
5.	413: 	"the whale"

============ 5 most frequent 3-grams

index	count	ngram
1.	116: 	"the sperm whale"
2.	109: 	"of the whale"
3.	88: 	"the white whale"
4.	64: 	"one of the"
5.	60: 	"of the sea"

============ 5 most frequent 4-grams

index	count	ngram
1.	43: 	"of the sperm whale"
2.	27: 	"the sperm whale s"
3.	20: 	"at the same time"
4.	18: 	"project gutenberg tm electronic"
5.	18: 	"of the whale s"

============ 5 most frequent 5-grams

index	count	ngram
1.	13: 	"the project gutenberg literary archive"
2.	13: 	"project gutenberg literary archive foundation"
3.	12: 	"project gutenberg tm electronic works"
4.	11: 	"of the sperm whale s"
5.	10: 	"and at the same time"
